In [2]:
# Libraries to manage data
import numpy as np
import pandas as pd

# Libraries for reqs and scrapping
import requests
from bs4 import BeautifulSoup

In [3]:
# Raw html response
covid_url = 'https://www.worldometers.info/coronavirus/'
req = requests.get(covid_url)
soup = BeautifulSoup(req.content, "html.parser")

In [4]:
# From table get what we want
# html tags: 
# variables names --> 'thead' --> 'tr' --> 'th'
# variables names --> 'tbody' --> 'tr' --> 'td'
variables = []
variables_html = soup.find_all('thead')[-1].find_all('tr')
for tag in variables_html:
    ths = tag.find_all(['th'])
    row = [th.text for th in ths]
    variables.append(row)
# variables

samples = []
samples_html = soup.find_all('tbody')[0].find_all('tr')
for tr in samples_html:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    samples.append(row)
# samples

df = pd.DataFrame(samples[:len(samples)-6], columns=variables[0]).drop(columns=["#"])       
df.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,\nNorth America\n,"20,340,889","+45,427","467,331","+1,336","12,341,255","+18,662","7,532,303","33,722",,,,,,North America,\n,,
1,\nAsia\n,"19,548,832","+56,868","318,960",+814,"17,777,467","+56,067","1,452,405","27,307",,,,,,Asia,\n,,
2,\nSouth America\n,"12,280,038","+11,430","345,686",+162,"10,858,312","+1,294","1,076,040","16,845",,,,,,South America,\n,,
3,\nEurope\n,"20,851,111","+142,343","482,138","+3,067","9,936,565","+110,533","10,432,408","26,568",,,,,,Europe,\n,,
4,\nAfrica\n,"2,467,835","+3,650","57,965",+55,"2,084,824","+3,321","325,046","2,758",,,,,,Africa,\n,,


In [5]:
df = df.iloc[8:,:-3].reset_index(drop=True)

In [6]:
# rename columns
df.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']
df = df[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]
df.sample(5)

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
47,Bulgaria,Europe,"6,923,955","188,288",,"6,339",,"93,728",,"88,221",567,"27,194",916,"1,090,431","157,487"
14,Iran,Asia,"84,488,279","1,145,651","+7,121","53,273",+178,"865,474","+8,961","226,904","5,627","13,560",631,"6,944,462","82,194"
163,San Marino,Europe,"33,964","2,049",,54,,"1,703",,292,5,"60,329","1,590","22,754","669,945"
58,Moldova,Europe,"4,029,590","133,360","+1,441","2,700",+26,"115,612","+1,382","15,048",264,"33,095",670,"522,523","129,672"
34,Saudi Arabia,Asia,"35,063,072","360,690",+174,"6,101",+10,"351,573",+208,"3,016",436,"10,287",174,"10,490,805","299,198"


In [7]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'

# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'
# addding 'WHO Region' column
df['WHO Region'] = df['Country/Region'].map(who_region)

# countries with missing "WHO Region"
df[df['WHO Region'].isna()]['Country/Region'].unique()
df.tail()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
209,Solomon Islands,Australia/Oceania,"694,564",17,,,,5,,12,,24,,"4,500","6,479",Western Pacific
210,Saint Pierre Miquelon,North America,"5,781",14,,,,14,,0,,"2,422",,"2,946","509,600",NaN
211,Montserrat,North America,"4,993",13,,1,,12,,0,,"2,604",200,626,"125,376",NaN
212,Western Sahara,Africa,"604,035",10,,1,,8,,1,,17,2,,,Africa
213,Anguilla,North America,"15,065",10,,,,10,,0,,664,,"4,043","268,370",NaN


In [8]:
# fix data
for col in df.columns[2:]:
    # replace comma with empty string
    df[col] = df[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df[col] = df[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df = df.replace('', np.nan)

# rename here what ever we want
df = df.rename(columns = {
    "Country/Region":"Country"
})
df = df[['Country','Population','TotalCases','TotalDeaths']]
df.sample(5)

,Country,Population,TotalCases,TotalDeaths
164,Chad,16639938,1818,102
85,Ghana,31369658,53553,331
124,Syria,17694535,9759,571
28,Pakistan,222857045,451494,9164
192,Caribbean Netherlands,26336,177,3


In [9]:
# save as .csv file
df.to_csv('covid19_data.csv', index=False)